## Import libraries

In [1]:
import pandas as pd
import spacy
from spacy.matcher import Matcher
from spacy import displacy

In [2]:
# Import English Library
nlp = spacy.load("en_core_web_sm")

## Load dataframe

In [3]:
df = pd.read_json("df_HCQ.json")
df.head()

,Publication ID,title,abstract,abstract_clean
0,pub.1126880632,COVID-19 and what pediatric rheumatologists sh...,"On March 11th, 2020 the World Health Organizat...","On March 11th, 2020 the World Health Organizat..."
1,pub.1127834352,Hydroxychloroquine or chloroquine with or with...,"BACKGROUND: Hydroxychloroquine or chloroquine,...","BACKGROUND: Hydroxychloroquine or chloroquine,..."
2,pub.1126667578,Hydroxychloroquine in patients mainly with mil...,Abstract Objectives To assess the efficacy and...,Abstract Objectives To assess the efficacy and...
3,pub.1125404383,Of chloroquine and COVID-19,Recent publications have brought attention to ...,Recent publications have brought attention to ...
4,pub.1127182972,An independent appraisal and re-analysis of hy...,A recent open-label study claimed that hydroxy...,A recent open-label study claimed that hydroxy...


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17 entries, 0 to 16
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Publication ID  17 non-null     object
 1   title           17 non-null     object
 2   abstract        17 non-null     object
 3   abstract_clean  17 non-null     object
dtypes: object(4)
memory usage: 680.0+ bytes


## Reshape dataframe

In [5]:
# Make new dataframe from 'df': df_HCQ
df_HCQ = df[["Publication ID", "title", "abstract_clean"]]

In [6]:
# Add column 'doc' to 'df_HCQ'
# 'df_HCQ["doc"]' shall contain Doc-objects made from abstracts
df_HCQ["doc"] = df_HCQ["abstract_clean"].apply(nlp)

In [7]:
df_HCQ.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17 entries, 0 to 16
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Publication ID  17 non-null     object
 1   title           17 non-null     object
 2   abstract_clean  17 non-null     object
 3   doc             17 non-null     object
dtypes: object(4)
memory usage: 680.0+ bytes


In [8]:
df_HCQ.head()

,Publication ID,title,abstract_clean,doc
0,pub.1126880632,COVID-19 and what pediatric rheumatologists sh...,"On March 11th, 2020 the World Health Organizat...","(On, March, 11th, ,, 2020, the, World, Health,..."
1,pub.1127834352,Hydroxychloroquine or chloroquine with or with...,"BACKGROUND: Hydroxychloroquine or chloroquine,...","(BACKGROUND, :, Hydroxychloroquine, or, chloro..."
2,pub.1126667578,Hydroxychloroquine in patients mainly with mil...,Abstract Objectives To assess the efficacy and...,"(Abstract, Objectives, To, assess, the, effica..."
3,pub.1125404383,Of chloroquine and COVID-19,Recent publications have brought attention to ...,"(Recent, publications, have, brought, attentio..."
4,pub.1127182972,An independent appraisal and re-analysis of hy...,A recent open-label study claimed that hydroxy...,"(A, recent, open, -, label, study, claimed, th..."


## Get last sentences of abstracts

In [9]:
def last_sents(doc):
    sents_list = [sent for sent in doc.sents]
    last_sents_list = sents_list[-4:]
    
    return last_sents_list

In [10]:
df_HCQ["last sentences"] = df_HCQ["doc"].apply(last_sents)
df_HCQ.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17 entries, 0 to 16
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Publication ID  17 non-null     object
 1   title           17 non-null     object
 2   abstract_clean  17 non-null     object
 3   doc             17 non-null     object
 4   last sentences  17 non-null     object
dtypes: object(5)
memory usage: 816.0+ bytes


In [11]:
df_HCQ.head(3)

,Publication ID,title,abstract_clean,doc,last sentences
0,pub.1126880632,COVID-19 and what pediatric rheumatologists sh...,"On March 11th, 2020 the World Health Organizat...","(On, March, 11th, ,, 2020, the, World, Health,...","[(For, patients, affected, by, rheumatic, dise..."
1,pub.1127834352,Hydroxychloroquine or chloroquine with or with...,"BACKGROUND: Hydroxychloroquine or chloroquine,...","(BACKGROUND, :, Hydroxychloroquine, or, chloro...","[(We, were, unable, to, confirm, a, benefit, o..."
2,pub.1126667578,Hydroxychloroquine in patients mainly with mil...,Abstract Objectives To assess the efficacy and...,"(Abstract, Objectives, To, assess, the, effica...","[((, note, :, if, posting, a, prospective, stu..."


In [12]:
sent_list_6 = df_HCQ["last sentences"].iloc[6]
sent_list_6

[24 vs 23 events, RR 1.14, 95% CI 0.65-2.00).,
 Eight patients receiving HCQ (9.5%) experienced electrocardiogram modifications requiring HCQ discontinuation.,
 Interpretation,
 These results do not support the use of HCQ in patients hospitalised for documented SARS-CoV-2-positive hypoxic pneumonia.]

In [13]:
sent_6 = sent_list_6[-1]
sent_6

These results do not support the use of HCQ in patients hospitalised for documented SARS-CoV-2-positive hypoxic pneumonia.

## Search for main verbs in last sentences

In [14]:
verbs_list = []

In [15]:
# Make and configure general matcher
matcher = Matcher(nlp.vocab)
pattern = [{"POS": "VERB"}]
matcher.add("verb_id", None, pattern)

In [16]:
def get_verbs(last_sentences):
    for sentence in last_sentences:
        matches = matcher(sentence)
        
        for verb_id, start, end in matches:
            verbs_list.append(sentence[start:end].lemma_)

In [17]:
df_HCQ["last sentences"].apply(get_verbs)

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
Name: last sentences, dtype: object

In [18]:
print(verbs_list)

['affect', 'relate', 'understand', 'issue', 'help', 'care', 'use', 'appear', 'promise', 'seem', 'drive', 'expect', 'play', 'treat', 'rheumatologist', 'take', 'offer', 'confirm', 'chloroquine', 'use', 'associate', 'decrease', 'increase', 'use', 'post', 'register', 'provide', 'explain', 'register', 'follow', 'upload', 'report', 'can', 'make', 'sign', 'should', 'direct', 'bring', 'use', 'infect', 'emerge', 'should', 'consider', 'claim', 'represent', 'promise', 'reveal', 'include', 'yield', 'show', 'regard', 'accord', 'observe', 'follow', 'should', 'take', 'achieve', 'receive', 'require', 'support', 'hospitalise', 'document', 'pair', 'suggest', 'should', 'recommend', 'force', 'halt', 'give', 'find', 'can', 'provide', 'update', 'may', 'may', 'mimic', 'may', 'need', 'understand', 'use', 'emerge', 'may', 'recommend', 'continue', 'modify', 'receive', 'chloroquine', 'support', 'use', 'give', 'test', 'assess', 'combat', 'chloroquine', 'approve', 'know', 'may', 'progress', 'severe', 'occur', 'cou

In [19]:
# Sort verbs in 'verbs_list' in alphabetical order
verbs_list.sort(key=str.lower)

In [20]:
print(verbs_list)

['accord', 'achieve', 'add', 'affect', 'appear', 'approve', 'approve', 'assess', 'associate', 'associate', 'bring', 'call', 'can', 'can', 'care', 'carry', 'cause', 'chloroquine', 'chloroquine', 'chloroquine', 'claim', 'combat', 'compare', 'conduct', 'confirm', 'consider', 'consider', 'continue', 'could', 'decrease', 'direct', 'discover', 'document', 'drive', 'emerge', 'emerge', 'endeavor', 'exchange', 'expect', 'explain', 'explore', 'face', 'find', 'follow', 'follow', 'force', 'give', 'give', 'give', 'halt', 'help', 'highlight', 'hospitalise', 'include', 'include', 'increase', 'infect', 'investigate', 'issue', 'issue', 'know', 'launch', 'like', 'make', 'may', 'may', 'may', 'may', 'may', 'mimic', 'modify', 'need', 'observe', 'occur', 'offer', 'ongoe', 'pair', 'play', 'post', 'progress', 'promise', 'promise', 'promote', 'provide', 'provide', 'publish', 'receive', 'receive', 'recommend', 'recommend', 'recommend', 'recommend', 'regard', 'register', 'register', 'reinforce', 'relate', 'repor

In [21]:
# Remove duplicates from 'verbs_list' (using some kind of workaround)
verbs_list = list(set(verbs_list))
# Sort verbs in 'verbs_list' in alphabetical order
verbs_list.sort(key=str.lower)
print(verbs_list)

['accord', 'achieve', 'add', 'affect', 'appear', 'approve', 'assess', 'associate', 'bring', 'call', 'can', 'care', 'carry', 'cause', 'chloroquine', 'claim', 'combat', 'compare', 'conduct', 'confirm', 'consider', 'continue', 'could', 'decrease', 'direct', 'discover', 'document', 'drive', 'emerge', 'endeavor', 'exchange', 'expect', 'explain', 'explore', 'face', 'find', 'follow', 'force', 'give', 'halt', 'help', 'highlight', 'hospitalise', 'include', 'increase', 'infect', 'investigate', 'issue', 'know', 'launch', 'like', 'make', 'may', 'mimic', 'modify', 'need', 'observe', 'occur', 'offer', 'ongoe', 'pair', 'play', 'post', 'progress', 'promise', 'promote', 'provide', 'publish', 'receive', 'recommend', 'regard', 'register', 'reinforce', 'relate', 'report', 'represent', 'require', 'reveal', 'rheumatologist', 'seem', 'severe', 'share', 'shorten', 'should', 'show', 'sign', 'spread', 'suggest', 'support', 'take', 'test', 'treat', 'trigger', 'understand', 'update', 'upload', 'use', 'would', 'yi

## Verbs expressing support for a statement

In [22]:
# List of "manually" chosen verbs: support_verbs
support_verbs = ['claim',
 'confirm',
 'provide',
 'reveal',
 'show',
 'suggest',
 'support',
 'yield']

In [23]:
# Make matcher to search for verbs that express support: matcher_support
matcher_support = Matcher(nlp.vocab)

for verb in support_verbs:
    pattern_support = [{"LEMMA": verb}]
    matcher_support.add(verb + "_id", None, pattern_support)

In [24]:
support_sentences_list = []

In [25]:
def support_sentences(last_sentences):
    for sentence in last_sentences:
        matches_support = matcher_support(sentence)
    
        for id, start, end in matches_support:
            support_sentences_list.append(sentence)

In [26]:
df_HCQ["last sentences"].apply(support_sentences)

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
Name: last sentences, dtype: object

In [27]:
# Remove duplicates from 'support_sentences_list'
support_sentences_list = list(set(support_sentences_list))

In [28]:
# Print sentences in 'support_sentences_list'
for i, sentence in enumerate(support_sentences_list):
    print(f"({i}) {sentence}")

(0) Twenty cases were treated in this study and showed a significant reduction of the viral carriage at D6-post inclusion compared to controls, and much lower average carrying duration than reported of untreated patients in the literature.
(1) Despite its small sample size our survey shows that hydroxychloroquine treatment is significantly associated with viral load reduction/disappearance in COVID-19 patients and its effect is reinforced by azithromycin.
(2) This systematic review and meta-analysis not only showed no clinical benefits regarding HCQ treatment with/without azithromycin for COVID-19 patients, but according to multiple sensitivity analysis, the higher mortality rates were observed for both HCQ and HCQ+AZM regimen groups, especially in the latter.
(3) (note: if posting a prospective study registered retrospectively, please provide a statement in the trial ID field explaining why the study was not registered in advance).
(4) These results do not support the use of HCQ in pa

## Expressions expressing negation of support for a statement

In [29]:
# List of sentences that express a critical objection to a statement: 'critical_objections'
critical_objections = support_sentences_list[:3]
critical_objections += support_sentences_list[6:7]
critical_objections += support_sentences_list[10:11]

In [30]:
for i, sentence in enumerate(critical_objections):
    print(f"({i}) {sentence}")

(0) Twenty cases were treated in this study and showed a significant reduction of the viral carriage at D6-post inclusion compared to controls, and much lower average carrying duration than reported of untreated patients in the literature.
(1) Despite its small sample size our survey shows that hydroxychloroquine treatment is significantly associated with viral load reduction/disappearance in COVID-19 patients and its effect is reinforced by azithromycin.
(2) This systematic review and meta-analysis not only showed no clinical benefits regarding HCQ treatment with/without azithromycin for COVID-19 patients, but according to multiple sensitivity analysis, the higher mortality rates were observed for both HCQ and HCQ+AZM regimen groups, especially in the latter.
(3) This re-analysis reveals severe limitations in the methodology of this study, including ambiguous inclusion/exclusion of participant data and inconsistent analysis techniques, and yielded nonsignificant differences between co

In [31]:
for sentence in critical_objections:
    print(sentence)
    displacy.render(sentence, style="dep", jupyter=True)
    print("----------\n")

Twenty cases were treated in this study and showed a significant reduction of the viral carriage at D6-post inclusion compared to controls, and much lower average carrying duration than reported of untreated patients in the literature.


----------

Despite its small sample size our survey shows that hydroxychloroquine treatment is significantly associated with viral load reduction/disappearance in COVID-19 patients and its effect is reinforced by azithromycin.


----------

This systematic review and meta-analysis not only showed no clinical benefits regarding HCQ treatment with/without azithromycin for COVID-19 patients, but according to multiple sensitivity analysis, the higher mortality rates were observed for both HCQ and HCQ+AZM regimen groups, especially in the latter.


----------

This re-analysis reveals severe limitations in the methodology of this study, including ambiguous inclusion/exclusion of participant data and inconsistent analysis techniques, and yielded nonsignificant differences between control and treatment groups across any treatment days.


----------

Given the enormous global push for the use of CQ for COVID-19, results such as the ones found in this trial can provide robust evidence for updated COVID-19 patient management recommendations.


----------

